In [154]:
import warnings
warnings.filterwarnings('ignore')
import os
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.datasets import fetch_20newsgroups, load_files

import pandas as pd
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Nems_list = ['gender', 'value_2', 'value_3', 'value_4', 'value_5', 'value_6', 'value_7', 'value_8', 'value_9', 'value_10', 'value_11', 'value_12', 'value_13', 'value_14', 'value_15','loan result',  ]
df = pd.read_csv('/Users/macbookpro/Desktop/Netology/2021/головаломка/crx.data.txt', names=Nems_list, na_values = "?")
df.head()

,gender,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10,value_11,value_12,value_13,value_14,value_15,loan result
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [155]:
df.isna().sum()

#разобрать пропуски и заменить и 

gender         12
value_2        12
value_3         0
value_4         6
value_5         6
value_6         9
value_7         9
value_8         0
value_9         0
value_10        0
value_11        0
value_12        0
value_13        0
value_14       13
value_15        0
loan result     0
dtype: int64

In [156]:
for i in df.columns:
    x = df[i].isna().mean()
    print(f' Пропусков в Столбце {i} {x : .2%} ' )

 Пропусков в Столбце gender  1.74% 
 Пропусков в Столбце value_2  1.74% 
 Пропусков в Столбце value_3  0.00% 
 Пропусков в Столбце value_4  0.87% 
 Пропусков в Столбце value_5  0.87% 
 Пропусков в Столбце value_6  1.30% 
 Пропусков в Столбце value_7  1.30% 
 Пропусков в Столбце value_8  0.00% 
 Пропусков в Столбце value_9  0.00% 
 Пропусков в Столбце value_10  0.00% 
 Пропусков в Столбце value_11  0.00% 
 Пропусков в Столбце value_12  0.00% 
 Пропусков в Столбце value_13  0.00% 
 Пропусков в Столбце value_14  1.88% 
 Пропусков в Столбце value_15  0.00% 
 Пропусков в Столбце loan result  0.00% 


In [157]:
# Пропусков в Столбце gender, value_2, value_4, value_5, value_6, value_7  Заменим соседним значение, это не повлияет на общее данные 
# Пропуски в столбце value_14 заменяем пустое хначение Модой
df.gender = df.gender.fillna(method='bfill')
df.value_2 = df.value_2.fillna(method='bfill')
df.value_4 = df.value_4.fillna(method='bfill')
df.value_5 = df.value_5.fillna(method='bfill')
df.value_6 = df.value_6.fillna(method='bfill')
df.value_7 = df.value_7.fillna(method='bfill')
df.value_14 = df.value_14.fillna(df.value_14.mode()[0])

In [158]:
for i in df.columns:
    x = df[i].isna().mean()
    print(f' Пропусков в Столбце {i} {x : .2%} ' )

 Пропусков в Столбце gender  0.00% 
 Пропусков в Столбце value_2  0.00% 
 Пропусков в Столбце value_3  0.00% 
 Пропусков в Столбце value_4  0.00% 
 Пропусков в Столбце value_5  0.00% 
 Пропусков в Столбце value_6  0.00% 
 Пропусков в Столбце value_7  0.00% 
 Пропусков в Столбце value_8  0.00% 
 Пропусков в Столбце value_9  0.00% 
 Пропусков в Столбце value_10  0.00% 
 Пропусков в Столбце value_11  0.00% 
 Пропусков в Столбце value_12  0.00% 
 Пропусков в Столбце value_13  0.00% 
 Пропусков в Столбце value_14  0.00% 
 Пропусков в Столбце value_15  0.00% 
 Пропусков в Столбце loan result  0.00% 


In [159]:
df_1= df[['gender', 'value_4', 'value_5', 'value_6', 'value_7', 'value_9', 'value_10', 'value_12', 'value_13']]
df_1

,gender,value_4,value_5,value_6,value_7,value_9,value_10,value_12,value_13
0,b,u,g,w,v,t,t,f,g
1,a,u,g,q,h,t,t,f,g
2,a,u,g,q,h,t,f,f,g
3,b,u,g,w,v,t,t,t,g
4,b,u,g,w,v,t,f,f,s
...,...,...,...,...,...,...,...,...,...
685,b,y,p,e,h,f,f,f,g
686,a,u,g,c,v,f,t,t,g
687,a,y,p,ff,ff,f,t,t,g
688,b,u,g,aa,v,f,f,f,g


In [160]:
df

,gender,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10,value_11,value_12,value_13,value_14,value_15,loan result
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260.0,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200.0,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200.0,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280.0,750,-


In [161]:
puls = df.dtypes[df.dtypes=='object'].index
puls

Index(['gender', 'value_4', 'value_5', 'value_6', 'value_7', 'value_9',
       'value_10', 'value_12', 'value_13', 'loan result'],
      dtype='object')

In [162]:
df_2 = df[['value_2', 'value_3', 'value_8', 'value_11', 'value_14', 'value_15']]
df_2

,value_2,value_3,value_8,value_11,value_14,value_15
0,30.83,0.000,1.25,1,202.0,0
1,58.67,4.460,3.04,6,43.0,560
2,24.50,0.500,1.50,0,280.0,824
3,27.83,1.540,3.75,5,100.0,3
4,20.17,5.625,1.71,0,120.0,0
...,...,...,...,...,...,...
685,21.08,10.085,1.25,0,260.0,0
686,22.67,0.750,2.00,2,200.0,394
687,25.25,13.500,2.00,1,200.0,1
688,17.92,0.205,0.04,0,280.0,750


In [163]:
df_x = pd.get_dummies(df, columns=['gender', 'value_4', 'value_5', 'value_6', 'value_7', 'value_9', 'value_10', 'value_12', 'value_13'])
df_x = df_x.drop('loan result', axis=1)
df_x

,value_2,value_3,value_8,value_11,value_14,value_15,gender_a,gender_b,value_4_l,value_4_u,...,value_7_z,value_9_f,value_9_t,value_10_f,value_10_t,value_12_f,value_12_t,value_13_g,value_13_p,value_13_s
0,30.83,0.000,1.25,1,202.0,0,0,1,0,1,...,0,0,1,0,1,1,0,1,0,0
1,58.67,4.460,3.04,6,43.0,560,1,0,0,1,...,0,0,1,0,1,1,0,1,0,0
2,24.50,0.500,1.50,0,280.0,824,1,0,0,1,...,0,0,1,1,0,1,0,1,0,0
3,27.83,1.540,3.75,5,100.0,3,0,1,0,1,...,0,0,1,0,1,0,1,1,0,0
4,20.17,5.625,1.71,0,120.0,0,0,1,0,1,...,0,0,1,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,21.08,10.085,1.25,0,260.0,0,0,1,0,0,...,0,1,0,1,0,1,0,1,0,0
686,22.67,0.750,2.00,2,200.0,394,1,0,0,1,...,0,1,0,0,1,0,1,1,0,0
687,25.25,13.500,2.00,1,200.0,1,1,0,0,0,...,0,1,0,0,1,0,1,1,0,0
688,17.92,0.205,0.04,0,280.0,750,0,1,0,1,...,0,1,0,1,0,1,0,1,0,0


In [164]:
df_y = df['loan result']
df_y

0      +
1      +
2      +
3      +
4      +
      ..
685    -
686    -
687    -
688    -
689    -
Name: loan result, Length: 690, dtype: object

In [165]:
#для Y применяем метод LabelEncoder

encoder_y = LabelEncoder()
y = encoder_y.fit_transform(df_y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [166]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, y, test_size=0.25)

In [167]:
from sklearn.linear_model import LogisticRegression

In [168]:
lr = LogisticRegression()

In [169]:
lr.fit(X_train, y_train)

LogisticRegression()

In [170]:
lr.predict(X_test)

array([1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1])

In [171]:
y_pred = lr.predict(X_test)
y_pred

array([1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1])

In [172]:
accuracy_score(y_test, y_pred)
               

0.884393063583815

In [173]:
X_test['predickt'] = y_pred 
X_test['test'] = y_test
X_test

,value_2,value_3,value_8,value_11,value_14,value_15,gender_a,gender_b,value_4_l,value_4_u,...,value_9_t,value_10_f,value_10_t,value_12_f,value_12_t,value_13_g,value_13_p,value_13_s,predickt,test
675,41.58,1.040,0.665,0,240.0,237,1,0,0,1,...,0,1,0,1,0,1,0,0,1,1
500,39.00,4.000,5.000,3,290.0,2279,0,1,0,1,...,1,0,1,0,1,1,0,0,0,0
207,28.67,9.335,5.665,6,381.0,168,0,1,0,1,...,1,0,1,1,0,1,0,0,0,0
520,20.42,7.500,1.500,1,160.0,234,1,0,0,1,...,1,0,1,1,0,1,0,0,0,0
408,16.00,3.125,0.085,1,0.0,6,0,1,0,1,...,0,0,1,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,59.50,2.750,1.750,5,60.0,58,0,1,0,1,...,1,0,1,0,1,1,0,0,0,0
232,33.50,1.750,4.500,4,253.0,857,0,1,0,1,...,1,0,1,0,1,1,0,0,0,0
174,22.67,10.500,1.335,0,100.0,0,0,1,0,1,...,1,1,0,1,0,1,0,0,0,0
607,32.33,3.500,0.500,0,232.0,0,0,1,0,1,...,0,1,0,0,1,1,0,0,1,1


In [174]:
from sklearn.metrics import confusion_matrix

results = confusion_matrix(y_test, y_pred)

In [175]:
results

array([[64,  8],
       [12, 89]])

In [176]:
#65 модель правильно определила кто отдатут кредит 
#86 правильно определила кто не отдадут кредит 
#17 не правильно определила что люди отдадут кредит, они не отдали  
#5 не правильно определила что люди не вернут кредит, они вернули